In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder


In [6]:
df = pd.read_csv('new_retail_data.csv')

print(df.info())
print(df.describe())
print(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302010 entries, 0 to 302009
Data columns (total 30 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Transaction_ID    301677 non-null  float64
 1   Customer_ID       301702 non-null  float64
 2   Name              301628 non-null  object 
 3   Email             301663 non-null  object 
 4   Phone             301648 non-null  float64
 5   Address           301695 non-null  object 
 6   City              301762 non-null  object 
 7   State             301729 non-null  object 
 8   Zipcode           301670 non-null  float64
 9   Country           301739 non-null  object 
 10  Age               301837 non-null  float64
 11  Gender            301693 non-null  object 
 12  Income            301720 non-null  object 
 13  Customer_Segment  301795 non-null  object 
 14  Date              301651 non-null  object 
 15  Year              301660 non-null  float64
 16  Month             30

In [16]:
print(df['products'].unique())

print(df['Product_Brand'].unique())

print(df['Product_Category'].unique())

print(df['Customer_Segment'].unique())

['Cycling shorts' 'Lenovo Tab' 'Sports equipment' 'Utility knife'
 'Chocolate cookies' 'QLED TV' 'Dress shirt' 'Dark chocolate' 'Candles'
 'Screwdriver set' 'Science' 'Bottled water' 'Fruit snacks' 'V-neck tee'
 'Drama' 'Flavored water' 'Samsung Galaxy Tab' 'Orange juice' 'Bookshelf'
 'Affogato' 'Business' 'Self-help' 'Sink' 'Grape juice' 'Stove' 'Huawei P'
 'Amazon Fire Tablet' 'Curtains' 'Pomegranate juice' 'Running shoes'
 'Political thriller' 'Google Pixel' 'Chocolate mousse' 'Vases'
 'Henley shirt' 'Toys' 'Iced tea' 'Apple juice' 'Microsoft Surface Laptop'
 'Desk' 'Bodycon dress' 'Sports headphones' 'Pendant lights' 'Clocks'
 'Comforter' 'Peacoat' 'Bed' 'White chocolate' 'Android TV'
 'Studio headphones' 'Sneakers' 'Sculptures' 'Cream soda' 'History'
 'Pliers' 'Oxfords' 'Short stories' 'Dining table' 'Cooking' 'Wall art'
 'Memoir' 'Maxi dress' 'LG Gram' 'Dishwasher' 'Cranberry juice' 'Crime'
 'Pillowcase set' 'Shower curtain' 'Samsung Galaxy' 'Curved TV'
 'DJ headphones' 'Button-d

UNDERSTAND THE TOTAL AMOUNTS PE

In [27]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')


#Normalize Categorical Columns
for c in ['products', 'Product_Brand', 'Product_Category', 'Product_Type']:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip().str.lower()


#normalize numerical columns
for c in['Amount', 'Total_Amount', 'Quantity', 'Total_Purchase']:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

amount = df.get('Amount')
qty = df.get('Quantity')
total = df.get('Total_Amount')

# Initialize sales as NaN
df['sales'] = np.nan

# Safely assign sales based on existing columns
if total is not None:
    df['sales'] = total
elif qty is not None and amount is not None:
    df['sales'] = qty * amount
elif amount is not None:
    df['sales'] = amount

df['YearMonth'] = df['Date'].dt.to_period('M').dt.to_timestamp()

print(df[['Date','YearMonth','Product_Brand','Product_Category','Product_Type','products','Amount','Total_Amount','sales']].head())
print(df['sales'].describe())

        Date  YearMonth  Product_Brand  ...      Amount Total_Amount        sales
0 2023-09-18 2023-09-01           nike  ...  108.028757   324.086270   324.086270
1 2023-12-31 2023-12-01        samsung  ...  403.353907   806.707815   806.707815
2 2023-04-26 2023-04-01  penguin books  ...  354.477600  1063.432799  1063.432799
3 2023-05-08 2023-05-01     home depot  ...  352.407717  2466.854021  2466.854021
4 2024-01-10 2024-01-01         nestle  ...  124.276524   248.553049   248.553049

[5 rows x 9 columns]
count    301660.000000
mean       1367.651156
std        1128.998515
min          10.003750
25%         438.724278
50%        1041.117547
75%        2029.999853
max        4999.625796
Name: sales, dtype: float64


In [ ]:
agg_cols = ['YearMonth', 'Product_Brand', 'Product_Category', 'Product_Type', 'products']

monthly = (df.dropna(subset=['YearMonth', 'sales']).groupby(agg_cols, dropna=False).agg(total_sales=('sales', 'sum'),
                                                                                        orders=('Transaction_ID', 'nunique'),
                                                                                        total_purchase=('Total_Purchases', 'sum')).reset_index())

top = (monthly.groupby(['Product_Brand', 'products'], as_index=False)['total_sales'].sum().sort_values(['Product_Brand', 'total_sales'], ascending=[True, False]))
display(top.head(20))


sample = monthly[(monthly['Product_Brand']=='brand_a') & (monthly['products']=='product_x')]